# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
K. Lee  ->  K. Lee  |  ['K. Lee']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']


Arxiv has 58 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2404.16085
extracting tarball to tmp_2404.16085...

 done.
Retrieving document from  https://arxiv.org/e-print/2404.16263
extracting tarball to tmp_2404.16263... done.
Retrieving document from  https://arxiv.org/e-print/2404.16488


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2404.16263/universe-2876451-english.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2404.16488... done.
Retrieving document from  https://arxiv.org/e-print/2404.16661


extracting tarball to tmp_2404.16661... done.
Retrieving document from  https://arxiv.org/e-print/2404.16669


extracting tarball to tmp_2404.16669... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.16488-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.16488) | **Two-Dimensional Eclipse Mapping of the Hot Jupiter WASP-43b with JWST  MIRI/LRS**  |
|| M. Hammond, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2024-04-26*|
|*Comments*| *Accepted for publication in The Astronomical Journal*|
|**Abstract**| We present eclipse maps of the two-dimensional thermal emission from the dayside of the hot Jupiter WASP-43b, derived from an observation of a phase curve with the JWST MIRI/LRS instrument. The observed eclipse shapes deviate significantly from those expected for a planet emitting uniformly over its surface. We fit a map to this deviation, constructed from spherical harmonics up to order $\ell_{\rm max}=2$, alongside the planetary, orbital, stellar, and systematic parameters. This yields a map with a meridionally-averaged eastward hot-spot shift of $(7.75 \pm 0.36)^{\circ}$, with no significant degeneracy between the map and the additional parameters. We show the latitudinal and longitudinal contributions of the day-side emission structure to the eclipse shape, finding a latitudinal signal of $\sim$200 ppm and a longitudinal signal of $\sim$250 ppm. To investigate the sensitivity of the map to the method, we fix the non-mapping parameters and derive an "eigenmap" fitted with an optimised number of orthogonal phase curves, which yields a similar map to the $\ell_{\rm max}=2$ map. We also fit a map up to $\ell_{\rm max}=3$, which shows a smaller hot-spot shift, with a larger uncertainty. These maps are similar to those produced by atmospheric simulations. We conclude that there is a significant mapping signal which constrains the spherical harmonic components of our model up to $\ell_{\rm max}=2$. Alternative mapping models may derive different structures with smaller-scale features; we suggest that further observations of WASP-43b and other planets will drive the development of more robust methods and more accurate maps. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.16085-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.16085) | **Cosmological constraints from weak lensing scattering transform using  HSC Y1 data**  |
|| S. Cheng, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-04-26*|
|*Comments*| *8 pages, 6 figures*|
|**Abstract**| As weak lensing surveys go deeper, there is an increasing need for reliable characterization of non-Gaussian structures at small angular scales. Here we present the first cosmological constraints with weak lensing scattering transform, a statistical estimator that combines efficiency, robustness, and interpretability. With the Hyper Suprime-Cam survey (HSC) year 1 data, we obtain $\Omega_\text{m}=0.29_{-0.03}^{+0.04}$, $S_8\equiv \sigma_8(\Omega_\text{m}/0.3)^{0.5}=0.83\pm0.02$, and intrinsic alignment strength $A_\text{IA}=1.0\pm0.4$ through simulation-based forward modeling. Our constraints are consistent with those derived from Planck. The error bar of $\Omega_\text{m}$ is 2 times tighter than that obtained from the power spectrum when the same scale range is used. This constraining power is on par with that of convolutional neural networks, suggesting that further investment in spatial information extraction may not yield substantial benefits. We also point out an internal tension of $S_8$ estimates linked to a redshift bin around z ~ 1 in the HSC data. We found that discarding that bin leads to a consistent decrease of $S_8$ from 0.83 to 0.79, for all statistical estimators. We argue that photometric redshift estimation is now the main limitation in the estimation of $S_8$ using HSC. This limitation is likely to affect other ground-based weak lensing surveys reaching redshifts greater than one. Alternative redshift estimation techniques, like clustering redshifts, may help alleviate this limitation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.16263-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.16263) | **New Timing Results of MSPs from NICER Observations**  |
|| S. Zheng, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2024-04-26*|
|*Comments*| **|
|**Abstract**| Millisecond pulsars (MSPs) are known for their long-term stability. Using six years of observations from the Neutron Star Interior Composition Explorer (NICER), we have conducted an in-depth analysis of the X-ray timing results for six MSPs: PSRs B1937+21, B1821$-$24, J0437$-$4715, J0030+0451, J0218+4232, and J2124$-$3358. The timing stability parameter $\sigma_z$ has been calculated, revealing remarkable timing precision on the order of $10^{-14}$ for PSRs B1937+21 and J0437$-$4715, and $10^{-13}$ for PSRs B1821$-$24, J0218+4232, and J0030+0451 over a timescale of 1000 days. These findings underscore the feasibility of autonomous in-orbit timekeeping using X-ray observations of MSPs. In addition, the consistency of long-term spin-down noise in the X-ray and radio bands has been investigated by comparison with IPTA radio data. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.16661-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.16661) | **Time-domain analysis of multi-waveband flares from AD Leonis**  |
|| <mark>X. Zhang</mark>, <mark>J. Li</mark>, Y. Gao, Q. Lei |
|*Appeared on*| *2024-04-26*|
|*Comments*| *Accepted for publication in MNRAS*|
|**Abstract**| Radio bursts of magnetically active stars reveal the intensity and activity of the stellar magnetic field. They may also be related to the planets around the stars. We monitored a radio-active star, AD Leonis, 3000 seconds per day for 17 days in November 2020, and 5000 seconds per day for 5 days in July 2023 with the Five-hundred-meter Aperture Spherical radio Telescope (FAST). Based on the simultaneous flux increases in Stokes I and Stokes V, one left-hand circular polarized radio burst is identified. The $\sim50\%$ degree of circular polarization indicates the burst being originated from non-thermal radiation related to the stellar magnetic field. Combining the newly discovered burst with previous observations of radio and X-ray bursts from AD Leonis, we did a periodicity analysis for the 49 bursts in total. No periodicity with confidence level $>3\sigma$ is found, while a candidate period of 3.04 days at $\approx 2\sigma$ confidence level is presented and discussed. Results of recent FAST observations and the periodicity analysis suggest a more compact campaign of observation toward this source, from which a more optimistic result of period search could be achieved. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.16669-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.16669) | **A candidate period of 4.605 day for FRB 20121102A and one possible  implication of its origin**  |
|| <mark>J. Li</mark>, Y. Gao, D. Li, K. Wu |
|*Appeared on*| *2024-04-26*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**| A firm establishment of the presence or the lack of periodicity in repeating Fast Radio Bursts (FRBs) is crucial for determining their origins. Here we compile 1145 radio bursts of FRB 20121102A with fluence larger than 0.15 Jy ms from observations using the Five-hundredmeter Aperture Spherical radio Telescope, Arecibo Observatory, Green Bank Telescope, Effelsberg Telescope, MeerKAT Telescope, Lovell Telescope, Deep Space Network 70 m radio telescopes, Very Large Array, and the Westerbork Synthesis Radio Telescope spanning the time interval of MJD 57175-58776. A quasi-period of $157.1_{-4.8}^{+5.2}$ day and a candidate quasi-period of $4.605_{-0.010}^{+0.003}$ day are found through the phase-folding probability binomial analysis. The former is consistent with previous findings and the latter is new. The 4.605 day periodicity is more obvious in high-energy bursts with fluence larger than $10^{38}$ erg. The presence of these (candidate) quasi-periods, together with the corresponding width of burst accumulation in the phase space, are consistent with the bursts' originating from a binary degenerate star system with a close-by planet around the primary neutron star. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

410  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
